<a href="https://colab.research.google.com/github/Nargesghsm/DS-Assignments/blob/main/IBM_Python_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **As a data scientist working for an investment firm, I will extract the revenue data for Tesla and GameStop and build a dashboard to compare the price of the stock vs the revenue**

Narges Ghasemi 24 September 2023

Question 1 - Extracting Tesla Stock Data Using yfinance - 2 Points


In [1]:
!pip install yfinance==0.1.67
!mamba install bs4==4.10.0 -y
!pip install nbformat==4.2.0
!pip install pandas==1.3.5
!pip install nbformat

/bin/bash: line 1: mamba: command not found


In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [4]:
Tesla = yf.Ticker('TSLA')

In [5]:
Tesla_data=Tesla.history(period="max")

In [6]:
Tesla_data.reset_index(inplace=True)
Tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29,1.266667,1.666667,1.169333,1.592667,281494500,0,0.0
1,2010-06-30,1.719333,2.028000,1.553333,1.588667,257806500,0,0.0
2,2010-07-01,1.666667,1.728000,1.351333,1.464000,123282000,0,0.0
3,2010-07-02,1.533333,1.540000,1.247333,1.280000,77097000,0,0.0
4,2010-07-06,1.333333,1.333333,1.055333,1.074000,103003500,0,0.0


Question 2 - Extracting Tesla Revenue Data Using Webscraping - 1 Points

In [7]:
url = 'https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue'
html_data = requests.get(url).text

In [8]:
#soup = BeautifulSoup(html_data,"html5lib")
soup = BeautifulSoup(html_data)

In [9]:
Tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])
#result = soup.find_all('table', {'class': 'historical_data_table table'})
for result in soup.find_all('table', attrs={'class':'historical_data_table table'}):
    if table.find('th').getText().startswith("Tesla Quarterly Revenue"):
        for row in result.find('tbody').find_all("tr"):
            col = row.find_all("td")
            if len(col) != 2: continue
            Date = col[0].text
            Revenue = col[1].text
            Tesla_revenue = Tesla_revenue.append({"Date":Date,  "Revenue":Revenue}, ignore_index=True)

In [10]:
Tesla_revenue.dropna(inplace=True)

Tesla_revenue = Tesla_revenue[Tesla_revenue['Revenue'] != ""]

In [11]:
Tesla_revenue.tail()

,Date,Revenue


Question 3 - Extracting GameStop Stock Data Using yfinance - 2 Points

In [12]:
gme = yf.Ticker("GME")

In [13]:
gme_data = gme.history(period='max')

In [14]:
gme_data.reset_index(inplace=True)
gme_data.head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14,1.712707,1.716073,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15,1.683251,1.687459,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


Question 4 - Extracting GameStop Revenue Data Using Webscraping - 1 Points

In [15]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html'
html_data = requests.get(url).text

In [16]:
#soup = BeautifulSoup(html_data,"html5lib")
soup = BeautifulSoup(html_data)

In [17]:
gme_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

for table in soup.find_all('table'):

    if ('GameStop Quarterly Revenue' in table.find('th').text):
        rows = table.find_all('tr')

        for row in rows:
            col = row.find_all('td')

            if col != []:
                date = col[0].text
                revenue = col[1].text.replace(',','').replace('$','')

                gme_revenue = gme_revenue.append({"Date":date, "Revenue":revenue}, ignore_index=True)

In [18]:
gme_revenue.tail()

,Date,Revenue
57,2006-01-31,1667
58,2005-10-31,534
59,2005-07-31,416
60,2005-04-30,475
61,2005-01-31,709


Question 5 - Tesla Stock and Revenue Dashboard - 2 Points

In [19]:
 Tesla_data.reset_index(inplace=True)

In [20]:
make_graph(Tesla_data[['Date','Close']], Tesla_revenue, 'Tesla')

ValueError: ignored

Question 6 - GameStop Stock and Revenue Dashboard- 2 Points

In [ ]:
make_graph(gme_data[['Date','Close']], gme_revenue, 'GameStop')

Question 7 - Sharing your Assignment Notebook - 2 Points